---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [4]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [114]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [94]:
ut = pd.read_table('university_towns.txt', header=None).rename(columns={0: 'RegionName'})
ut

,RegionName
0,Alabama[edit]
1,Auburn (Auburn University)[1]
2,Florence (University of North Alabama)
3,Jacksonville (Jacksonville State University)[2]
4,Livingston (University of West Alabama)[2]
5,Montevallo (University of Montevallo)[2]
6,Troy (Troy University)[2]
7,"Tuscaloosa (University of Alabama, Stillman Co..."
8,Tuskegee (Tuskegee University)[5]
9,Alaska[edit]


In [95]:
# 50个州
ut.RegionName.str.match('[\w\s]*\[edit\]').sum()

50

In [97]:
y = ''
def add_states(x):
    # 最后六个字符是[edit]的话，那么就是state;
    # 否则均是region，无论有没有()
    global y
    if x[-6:] == '[edit]':
        y = x[: -6]
        return np.NaN
    else:
        return y

ut['State'] = ut.RegionName.apply(add_states)
ut = ut.dropna().reindex(columns=['State', 'RegionName'])
ut

,State,RegionName
1,Alabama,Auburn (Auburn University)[1]
2,Alabama,Florence (University of North Alabama)
3,Alabama,Jacksonville (Jacksonville State University)[2]
4,Alabama,Livingston (University of West Alabama)[2]
5,Alabama,Montevallo (University of Montevallo)[2]
6,Alabama,Troy (Troy University)[2]
7,Alabama,"Tuscaloosa (University of Alabama, Stillman Co..."
8,Alabama,Tuskegee (Tuskegee University)[5]
10,Alaska,Fairbanks (University of Alaska Fairbanks)[2]
12,Arizona,Flagstaff (Northern Arizona University)[6]


In [98]:
ut.RegionName = ut.RegionName.str.replace('\s*\([\d\s\w\,\[\]\)\–\-\']*', '', regex=True)
ut

,State,RegionName
1,Alabama,Auburn
2,Alabama,Florence
3,Alabama,Jacksonville
4,Alabama,Livingston
5,Alabama,Montevallo
6,Alabama,Troy
7,Alabama,Tuscaloosa
8,Alabama,Tuskegee
10,Alaska,Fairbanks
12,Arizona,Flagstaff


In [2]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    ut = pd.read_table('university_towns.txt', header=None).rename(columns={0: 'RegionName'})
    y = ''
    def add_states(x):
        # 最后六个字符是[edit]的话，那么就是state;
        # 否则均是region，无论有没有()
        global y
        if x[-6:] == '[edit]':
            y = x[: -6]
            return np.NaN
        else:
            return y

    ut['State'] = ut.RegionName.apply(add_states)
    ut = ut.dropna().reindex(columns=['State', 'RegionName'])
    ut.RegionName = ut.RegionName.str.replace('\s*\([\d\s\w\,\[\]\)\–\-\']*', '', regex=True)
    return ut

get_list_of_university_towns()

,State,RegionName
1,Alabama,Auburn
2,Alabama,Florence
3,Alabama,Jacksonville
4,Alabama,Livingston
5,Alabama,Montevallo
6,Alabama,Troy
7,Alabama,Tuscaloosa
8,Alabama,Tuskegee
10,Alaska,Fairbanks
12,Arizona,Flagstaff


In [21]:
yq = pd.read_excel('gdplev.xls', header=5)[['Unnamed: 3', 'GDP in billions of current dollars.1', 'GDP in billions of chained 2009 dollars.1']].rename(columns={'Unnamed: 3': 'Quarter'})
yq = yq.set_index('Quarter')['2000q1':]
yq

,GDP in billions of current dollars.1,GDP in billions of chained 2009 dollars.1
Quarter,,
2000q1,10031.0,12359.1
2000q2,10278.3,12592.5
2000q3,10357.4,12607.7
2000q4,10472.3,12679.3
2001q1,10508.1,12643.3
2001q2,10638.4,12710.3
2001q3,10639.5,12670.1
2001q4,10701.3,12705.3
2002q1,10834.4,12822.3


In [39]:
length = len(yq)
temp = yq['GDP in billions of chained 2009 dollars.1']
begin, end, flag = -1, -1, False
for i in range(1, length):
    if i+1<length:
        if not flag and temp[i-1]>temp[i] and temp[i]>temp[i+1]:
            begin = temp.index[i]
            flag = True
        if flag and temp[i]>temp[i-1] and temp[i+1]>temp[i]:
            end = temp.index[i]
            flag = False
begin    

'2008q3'

In [44]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    
    # begin: 第一个下降的季度
    yq = pd.read_excel('gdplev.xls', header=5)[['Unnamed: 3', 'GDP in billions of current dollars.1', 'GDP in billions of chained 2009 dollars.1']].rename(columns={'Unnamed: 3': 'Quarter'})
    yq = yq.set_index('Quarter')['2000q1':]
    length = len(yq)
    temp = yq['GDP in billions of chained 2009 dollars.1']
    begin, end, flag = -1, -1, False
    for i in range(1, length):
        if i+1<length:
            if not flag and temp[i-1]>temp[i] and temp[i]>temp[i+1]:
                begin = temp.index[i]
                flag = True
            if flag and temp[i]>temp[i-1] and temp[i+1]>temp[i]:
                end = temp.index[i+1]
                flag = False
    return begin

get_recession_start()

'2008q3'

In [45]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
       
    # end: 最后一个增长的季度
    yq = pd.read_excel('gdplev.xls', header=5)[['Unnamed: 3', 'GDP in billions of current dollars.1', 'GDP in billions of chained 2009 dollars.1']].rename(columns={'Unnamed: 3': 'Quarter'})
    yq = yq.set_index('Quarter')['2000q1':]
    length = len(yq)
    temp = yq['GDP in billions of chained 2009 dollars.1']
    begin, end, flag = -1, -1, False
    for i in range(1, length):
        if i+1<length:
            if not flag and temp[i-1]>temp[i] and temp[i]>temp[i+1]:
                begin = temp.index[i]
                flag = True
            if flag and temp[i]>temp[i-1] and temp[i+1]>temp[i]:
                end = temp.index[i+1]
                flag = False
    return end

get_recession_end()

'2009q4'

In [47]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    
    begin = get_recession_start()
    end = get_recession_end()
    df = yq.loc[begin: end]
    return df['GDP in billions of chained 2009 dollars.1'].idxmin()

get_recession_bottom()

'2009q2'

In [89]:
col_names = []
for i in range(2000, 2017):
    for j in range(1, 10):
        col_names.append(str(i)+'-0'+str(j))
    for j in range(10, 13):
        col_names.append(str(i)+'-'+str(j))  
col_names = col_names[: -4]
hs = pd.read_csv('City_Zhvi_AllHomes.csv', index_col=["State","RegionName"])[col_names]
hs

,,2000-01,2000-02,2000-03,2000-04,2000-05,2000-06,2000-07,2000-08,2000-09,2000-10,...,2015-11,2015-12,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08
State,RegionName,,,,,,,,,,,,,,,,,,,,,
NY,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,573600,576200,578400,582200,588000,592200,592500,590200,588000,586400
CA,Los Angeles,204400.0,207000.0,209800.0,212300.0,214500.0,216600.0,219000.0,221100.0,222800.0,224300.0,...,558200,560800,562800,565600,569700,574000,577800,580600,583000,585100
IL,Chicago,136800.0,138300.0,140100.0,141900.0,143700.0,145300.0,146700.0,147900.0,149000.0,150400.0,...,207800,206900,206200,205800,206200,207300,208200,209100,211000,213000
PA,Philadelphia,52700.0,53100.0,53200.0,53400.0,53700.0,53800.0,53800.0,54100.0,54500.0,54700.0,...,122300,121600,121800,123300,125200,126400,127000,127400,128300,129100
AZ,Phoenix,111000.0,111700.0,112800.0,113700.0,114300.0,115100.0,115600.0,115900.0,116500.0,117200.0,...,183800,185300,186600,188000,189100,190200,191300,192800,194500,195900
NV,Las Vegas,131700.0,132600.0,133500.0,134100.0,134400.0,134600.0,134800.0,135400.0,136000.0,136400.0,...,190600,192000,193600,194800,195400,196100,197300,198200,199300,200600
CA,San Diego,219200.0,222900.0,226600.0,230200.0,234400.0,238500.0,242000.0,245300.0,249000.0,252500.0,...,525700,526700,527800,529200,531000,533900,536900,537900,539000,540500
TX,Dallas,85100.0,84500.0,83800.0,83600.0,83800.0,84200.0,84500.0,84900.0,85200.0,86000.0,...,134600,136600,138700,140600,142200,143300,144500,146000,148200,150400
CA,San Jose,364100.0,374000.0,384700.0,395700.0,407100.0,416900.0,424700.0,431700.0,439200.0,447600.0,...,789700,792100,795800,803100,811900,817600,819100,820100,821700,822700


In [90]:
i, j = 0, 1
while i < len(col_names):
    temp = col_names[i: i+3]
    name = col_names[i][: 4] + 'q' + str(j)
    # print(name)
    hs[name] = hs[temp].apply(lambda x: np.mean(x), axis=1)
    i = i+3
    if j+1 > 4:
        j = 1
    else:
        j = j+1
hs

,,2000-01,2000-02,2000-03,2000-04,2000-05,2000-06,2000-07,2000-08,2000-09,2000-10,...,2014q2,2014q3,2014q4,2015q1,2015q2,2015q3,2015q4,2016q1,2016q2,2016q3
State,RegionName,,,,,,,,,,,,,,,,,,,,,
NY,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.154667e+05,5.228000e+05,5.280667e+05,5.322667e+05,5.408000e+05,5.572000e+05,5.728333e+05,5.828667e+05,5.916333e+05,587200.0
CA,Los Angeles,204400.0,207000.0,209800.0,212300.0,214500.0,216600.0,219000.0,221100.0,222800.0,224300.0,...,4.980333e+05,5.090667e+05,5.188667e+05,5.288000e+05,5.381667e+05,5.472667e+05,5.577333e+05,5.660333e+05,5.774667e+05,584050.0
IL,Chicago,136800.0,138300.0,140100.0,141900.0,143700.0,145300.0,146700.0,147900.0,149000.0,150400.0,...,1.926333e+05,1.957667e+05,2.012667e+05,2.010667e+05,2.060333e+05,2.083000e+05,2.079000e+05,2.060667e+05,2.082000e+05,212000.0
PA,Philadelphia,52700.0,53100.0,53200.0,53400.0,53700.0,53800.0,53800.0,54100.0,54500.0,54700.0,...,1.137333e+05,1.153000e+05,1.156667e+05,1.162000e+05,1.179667e+05,1.212333e+05,1.222000e+05,1.234333e+05,1.269333e+05,128700.0
AZ,Phoenix,111000.0,111700.0,112800.0,113700.0,114300.0,115100.0,115600.0,115900.0,116500.0,117200.0,...,1.642667e+05,1.653667e+05,1.685000e+05,1.715333e+05,1.741667e+05,1.790667e+05,1.838333e+05,1.879000e+05,1.914333e+05,195200.0
NV,Las Vegas,131700.0,132600.0,133500.0,134100.0,134400.0,134600.0,134800.0,135400.0,136000.0,136400.0,...,1.700667e+05,1.734000e+05,1.754667e+05,1.775000e+05,1.816000e+05,1.867667e+05,1.906333e+05,1.946000e+05,1.972000e+05,199950.0
CA,San Diego,219200.0,222900.0,226600.0,230200.0,234400.0,238500.0,242000.0,245300.0,249000.0,252500.0,...,4.802000e+05,4.890333e+05,4.964333e+05,5.033667e+05,5.120667e+05,5.197667e+05,5.254667e+05,5.293333e+05,5.362333e+05,539750.0
TX,Dallas,85100.0,84500.0,83800.0,83600.0,83800.0,84200.0,84500.0,84900.0,85200.0,86000.0,...,1.066333e+05,1.089000e+05,1.115333e+05,1.137000e+05,1.211333e+05,1.285667e+05,1.346000e+05,1.405000e+05,1.446000e+05,149300.0
CA,San Jose,364100.0,374000.0,384700.0,395700.0,407100.0,416900.0,424700.0,431700.0,439200.0,447600.0,...,6.794000e+05,6.970333e+05,7.149333e+05,7.314333e+05,7.567333e+05,7.764000e+05,7.891333e+05,8.036000e+05,8.189333e+05,822200.0


In [91]:
hs = hs.drop(col_names, axis=1)
hs

,,2000q1,2000q2,2000q3,2000q4,2001q1,2001q2,2001q3,2001q4,2002q1,2002q2,...,2014q2,2014q3,2014q4,2015q1,2015q2,2015q3,2015q4,2016q1,2016q2,2016q3
State,RegionName,,,,,,,,,,,,,,,,,,,,,
NY,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.154667e+05,5.228000e+05,5.280667e+05,5.322667e+05,5.408000e+05,5.572000e+05,5.728333e+05,5.828667e+05,5.916333e+05,587200.0
CA,Los Angeles,2.070667e+05,2.144667e+05,2.209667e+05,2.261667e+05,2.330000e+05,2.391000e+05,2.450667e+05,2.530333e+05,2.619667e+05,2.727000e+05,...,4.980333e+05,5.090667e+05,5.188667e+05,5.288000e+05,5.381667e+05,5.472667e+05,5.577333e+05,5.660333e+05,5.774667e+05,584050.0
IL,Chicago,1.384000e+05,1.436333e+05,1.478667e+05,1.521333e+05,1.569333e+05,1.618000e+05,1.664000e+05,1.704333e+05,1.755000e+05,1.775667e+05,...,1.926333e+05,1.957667e+05,2.012667e+05,2.010667e+05,2.060333e+05,2.083000e+05,2.079000e+05,2.060667e+05,2.082000e+05,212000.0
PA,Philadelphia,5.300000e+04,5.363333e+04,5.413333e+04,5.470000e+04,5.533333e+04,5.553333e+04,5.626667e+04,5.753333e+04,5.913333e+04,6.073333e+04,...,1.137333e+05,1.153000e+05,1.156667e+05,1.162000e+05,1.179667e+05,1.212333e+05,1.222000e+05,1.234333e+05,1.269333e+05,128700.0
AZ,Phoenix,1.118333e+05,1.143667e+05,1.160000e+05,1.174000e+05,1.196000e+05,1.215667e+05,1.227000e+05,1.243000e+05,1.265333e+05,1.283667e+05,...,1.642667e+05,1.653667e+05,1.685000e+05,1.715333e+05,1.741667e+05,1.790667e+05,1.838333e+05,1.879000e+05,1.914333e+05,195200.0
NV,Las Vegas,1.326000e+05,1.343667e+05,1.354000e+05,1.370000e+05,1.395333e+05,1.417333e+05,1.433667e+05,1.461333e+05,1.493333e+05,1.509333e+05,...,1.700667e+05,1.734000e+05,1.754667e+05,1.775000e+05,1.816000e+05,1.867667e+05,1.906333e+05,1.946000e+05,1.972000e+05,199950.0
CA,San Diego,2.229000e+05,2.343667e+05,2.454333e+05,2.560333e+05,2.672000e+05,2.762667e+05,2.845000e+05,2.919333e+05,3.012333e+05,3.128667e+05,...,4.802000e+05,4.890333e+05,4.964333e+05,5.033667e+05,5.120667e+05,5.197667e+05,5.254667e+05,5.293333e+05,5.362333e+05,539750.0
TX,Dallas,8.446667e+04,8.386667e+04,8.486667e+04,8.783333e+04,8.973333e+04,8.930000e+04,8.906667e+04,9.090000e+04,9.256667e+04,9.380000e+04,...,1.066333e+05,1.089000e+05,1.115333e+05,1.137000e+05,1.211333e+05,1.285667e+05,1.346000e+05,1.405000e+05,1.446000e+05,149300.0
CA,San Jose,3.742667e+05,4.065667e+05,4.318667e+05,4.555000e+05,4.706667e+05,4.702000e+05,4.568000e+05,4.455667e+05,4.414333e+05,4.577667e+05,...,6.794000e+05,6.970333e+05,7.149333e+05,7.314333e+05,7.567333e+05,7.764000e+05,7.891333e+05,8.036000e+05,8.189333e+05,822200.0


In [5]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    
    col_names = []
    for i in range(2000, 2017):
        for j in range(1, 10):
            col_names.append(str(i)+'-0'+str(j))
        for j in range(10, 13):
            col_names.append(str(i)+'-'+str(j))  
    col_names = col_names[: -4]
    hs = pd.read_csv('City_Zhvi_AllHomes.csv', index_col=["State","RegionName"])[col_names]
    i, j = 0, 1
    while i < len(col_names):
        temp = col_names[i: i+3]
        name = col_names[i][: 4] + 'q' + str(j)
        # print(name)
        hs[name] = hs[temp].apply(lambda x: np.mean(x), axis=1)
        i = i+3
        if j+1 > 4:
            j = 1
        else:
            j = j+1
    hs = hs.drop(col_names, axis=1)
    return hs

convert_housing_data_to_quarters()

,,2000q1,2000q2,2000q3,2000q4,2001q1,2001q2,2001q3,2001q4,2002q1,2002q2,...,2014q2,2014q3,2014q4,2015q1,2015q2,2015q3,2015q4,2016q1,2016q2,2016q3
State,RegionName,,,,,,,,,,,,,,,,,,,,,
NY,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.154667e+05,5.228000e+05,5.280667e+05,5.322667e+05,5.408000e+05,5.572000e+05,5.728333e+05,5.828667e+05,5.916333e+05,587200.0
CA,Los Angeles,2.070667e+05,2.144667e+05,2.209667e+05,2.261667e+05,2.330000e+05,2.391000e+05,2.450667e+05,2.530333e+05,2.619667e+05,2.727000e+05,...,4.980333e+05,5.090667e+05,5.188667e+05,5.288000e+05,5.381667e+05,5.472667e+05,5.577333e+05,5.660333e+05,5.774667e+05,584050.0
IL,Chicago,1.384000e+05,1.436333e+05,1.478667e+05,1.521333e+05,1.569333e+05,1.618000e+05,1.664000e+05,1.704333e+05,1.755000e+05,1.775667e+05,...,1.926333e+05,1.957667e+05,2.012667e+05,2.010667e+05,2.060333e+05,2.083000e+05,2.079000e+05,2.060667e+05,2.082000e+05,212000.0
PA,Philadelphia,5.300000e+04,5.363333e+04,5.413333e+04,5.470000e+04,5.533333e+04,5.553333e+04,5.626667e+04,5.753333e+04,5.913333e+04,6.073333e+04,...,1.137333e+05,1.153000e+05,1.156667e+05,1.162000e+05,1.179667e+05,1.212333e+05,1.222000e+05,1.234333e+05,1.269333e+05,128700.0
AZ,Phoenix,1.118333e+05,1.143667e+05,1.160000e+05,1.174000e+05,1.196000e+05,1.215667e+05,1.227000e+05,1.243000e+05,1.265333e+05,1.283667e+05,...,1.642667e+05,1.653667e+05,1.685000e+05,1.715333e+05,1.741667e+05,1.790667e+05,1.838333e+05,1.879000e+05,1.914333e+05,195200.0
NV,Las Vegas,1.326000e+05,1.343667e+05,1.354000e+05,1.370000e+05,1.395333e+05,1.417333e+05,1.433667e+05,1.461333e+05,1.493333e+05,1.509333e+05,...,1.700667e+05,1.734000e+05,1.754667e+05,1.775000e+05,1.816000e+05,1.867667e+05,1.906333e+05,1.946000e+05,1.972000e+05,199950.0
CA,San Diego,2.229000e+05,2.343667e+05,2.454333e+05,2.560333e+05,2.672000e+05,2.762667e+05,2.845000e+05,2.919333e+05,3.012333e+05,3.128667e+05,...,4.802000e+05,4.890333e+05,4.964333e+05,5.033667e+05,5.120667e+05,5.197667e+05,5.254667e+05,5.293333e+05,5.362333e+05,539750.0
TX,Dallas,8.446667e+04,8.386667e+04,8.486667e+04,8.783333e+04,8.973333e+04,8.930000e+04,8.906667e+04,9.090000e+04,9.256667e+04,9.380000e+04,...,1.066333e+05,1.089000e+05,1.115333e+05,1.137000e+05,1.211333e+05,1.285667e+05,1.346000e+05,1.405000e+05,1.446000e+05,149300.0
CA,San Jose,3.742667e+05,4.065667e+05,4.318667e+05,4.555000e+05,4.706667e+05,4.702000e+05,4.568000e+05,4.455667e+05,4.414333e+05,4.577667e+05,...,6.794000e+05,6.970333e+05,7.149333e+05,7.314333e+05,7.567333e+05,7.764000e+05,7.891333e+05,8.036000e+05,8.189333e+05,822200.0


In [7]:
convert_housing_data_to_quarters()['2016q3'].isnull().sum()

0

In [147]:
hd = convert_housing_data_to_quarters()

In [148]:
columns = hd.columns
start = '2008q3' # get_recession_start()
bottom = '2009q2' # get_recession_bottom()
i, j = -1, -1
for x in range(0, len(columns)):
    if columns[x] == start:
        i = x
    if columns[x] == bottom:
        j = x
cols = columns[i: j+1]
cols
hds = hd[cols]
hds

,,2008q3,2008q4,2009q1,2009q2
State,RegionName,,,,
NY,New York,4.997667e+05,4.879333e+05,4.777333e+05,4.658333e+05
CA,Los Angeles,4.695000e+05,4.439667e+05,4.262667e+05,4.139000e+05
IL,Chicago,2.320000e+05,2.270333e+05,2.237667e+05,2.197000e+05
PA,Philadelphia,1.169333e+05,1.158667e+05,1.162000e+05,1.161667e+05
AZ,Phoenix,1.937667e+05,1.833333e+05,1.775667e+05,1.682333e+05
NV,Las Vegas,2.133667e+05,1.949333e+05,1.812000e+05,1.643333e+05
CA,San Diego,4.246667e+05,4.076333e+05,3.957000e+05,3.895000e+05
TX,Dallas,1.121667e+05,1.099000e+05,1.076667e+05,1.051000e+05
CA,San Jose,5.831333e+05,5.601000e+05,5.481667e+05,5.303000e+05


In [149]:
hds = hds.reset_index()
hds.State = hds.State.map(states)
hds = hds.set_index(['State', 'RegionName'])
hds.head()

,,2008q3,2008q4,2009q1,2009q2
State,RegionName,,,,
New York,New York,499766.666667,487933.333333,477733.333333,465833.333333
California,Los Angeles,469500.000000,443966.666667,426266.666667,413900.000000
Illinois,Chicago,232000.000000,227033.333333,223766.666667,219700.000000
Pennsylvania,Philadelphia,116933.333333,115866.666667,116200.000000,116166.666667
Arizona,Phoenix,193766.666667,183333.333333,177566.666667,168233.333333


In [150]:
ut = get_list_of_university_towns()
ut.head()

,State,RegionName
1,Alabama,Auburn
2,Alabama,Florence
3,Alabama,Jacksonville
4,Alabama,Livingston
5,Alabama,Montevallo


In [151]:
ut_price = pd.merge(hds, ut, left_index=True, right_on=['State', 'RegionName']).set_index(['State', 'RegionName'])
ut_price

,,2008q3,2008q4,2009q1,2009q2
State,RegionName,,,,
Nevada,Las Vegas,213366.666667,194933.333333,181200.000000,164333.333333
California,San Diego,424666.666667,407633.333333,395700.000000,389500.000000
Texas,Dallas,112166.666667,109900.000000,107666.666667,105100.000000
Ohio,Columbus,112200.000000,111600.000000,110966.666667,109766.666667
Tennessee,Memphis,83700.000000,80900.000000,77966.666667,76000.000000
Massachusetts,Boston,347333.333333,341133.333333,328633.333333,325200.000000
Tennessee,Nashville,154333.333333,154200.000000,152400.000000,150133.333333
Wisconsin,Milwaukee,127900.000000,124500.000000,120600.000000,116733.333333
Arizona,Tucson,175766.666667,172133.333333,168866.666667,164366.666667


In [152]:
ut_price['Growth'] = ut_price['2009q2'] - ut_price['2008q3']
ut_price

,,2008q3,2008q4,2009q1,2009q2,Growth
State,RegionName,,,,,
Nevada,Las Vegas,213366.666667,194933.333333,181200.000000,164333.333333,-49033.333333
California,San Diego,424666.666667,407633.333333,395700.000000,389500.000000,-35166.666667
Texas,Dallas,112166.666667,109900.000000,107666.666667,105100.000000,-7066.666667
Ohio,Columbus,112200.000000,111600.000000,110966.666667,109766.666667,-2433.333333
Tennessee,Memphis,83700.000000,80900.000000,77966.666667,76000.000000,-7700.000000
Massachusetts,Boston,347333.333333,341133.333333,328633.333333,325200.000000,-22133.333333
Tennessee,Nashville,154333.333333,154200.000000,152400.000000,150133.333333,-4200.000000
Wisconsin,Milwaukee,127900.000000,124500.000000,120600.000000,116733.333333,-11166.666667
Arizona,Tucson,175766.666667,172133.333333,168866.666667,164366.666667,-11400.000000


In [153]:
nut_price = hds.drop(ut_price.index, axis=0)
nut_price

,,2008q3,2008q4,2009q1,2009q2
State,RegionName,,,,
New York,New York,4.997667e+05,4.879333e+05,4.777333e+05,4.658333e+05
California,Los Angeles,4.695000e+05,4.439667e+05,4.262667e+05,4.139000e+05
Illinois,Chicago,2.320000e+05,2.270333e+05,2.237667e+05,2.197000e+05
Pennsylvania,Philadelphia,1.169333e+05,1.158667e+05,1.162000e+05,1.161667e+05
Arizona,Phoenix,1.937667e+05,1.833333e+05,1.775667e+05,1.682333e+05
California,San Jose,5.831333e+05,5.601000e+05,5.481667e+05,5.303000e+05
Florida,Jacksonville,1.547333e+05,1.491667e+05,1.452667e+05,1.408333e+05
California,San Francisco,7.543667e+05,7.403667e+05,7.182333e+05,6.950000e+05
Texas,Austin,2.110333e+05,2.074667e+05,2.070000e+05,2.040000e+05


In [154]:
nut_price['Growth'] = nut_price['2009q2'] - nut_price['2008q3']
nut_price

/Users/zhujun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,,2008q3,2008q4,2009q1,2009q2,Growth
State,RegionName,,,,,
New York,New York,4.997667e+05,4.879333e+05,4.777333e+05,4.658333e+05,-33933.333333
California,Los Angeles,4.695000e+05,4.439667e+05,4.262667e+05,4.139000e+05,-55600.000000
Illinois,Chicago,2.320000e+05,2.270333e+05,2.237667e+05,2.197000e+05,-12300.000000
Pennsylvania,Philadelphia,1.169333e+05,1.158667e+05,1.162000e+05,1.161667e+05,-766.666667
Arizona,Phoenix,1.937667e+05,1.833333e+05,1.775667e+05,1.682333e+05,-25533.333333
California,San Jose,5.831333e+05,5.601000e+05,5.481667e+05,5.303000e+05,-52833.333333
Florida,Jacksonville,1.547333e+05,1.491667e+05,1.452667e+05,1.408333e+05,-13900.000000
California,San Francisco,7.543667e+05,7.403667e+05,7.182333e+05,6.950000e+05,-59366.666667
Texas,Austin,2.110333e+05,2.074667e+05,2.070000e+05,2.040000e+05,-7033.333333


In [157]:
from scipy import stats

stats.ttest_ind(ut_price['Growth'], nut_price['Growth'], nan_policy='omit')

Ttest_indResult(statistic=2.726006334708476, pvalue=0.006421874144769314)

In [158]:
ut_price.Growth.mean()

-8847.414965986398

In [159]:
nut_price.Growth.mean()

-13082.214614240776

In [160]:
print(True, 0.006421874144769314, 'university town')

True 0.006421874144769314 university town


In [161]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    return (True, 0.006421874144769314, 'university town')

run_ttest()

(True, 0.006421874144769314, 'university town')